<a href="https://colab.research.google.com/github/mrcruz117/linkedin-silverkite-forecast-templates/blob/main/LinkedIn_Silverkite_Parameter_Tuning_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Silverkite Parameter Tuning

In [1]:
#change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [3]:
# !pip install greykite

In [5]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from greykite.framework.templates.autogen.forecast_config import *
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.common.features.timeseries_features import *
from greykite.common.evaluation import EvaluationMetricEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from plotly.offline import iplot

In [9]:
#load the data
#YYYY-MM-DD
df = pd.read_csv('nyc_data.csv')
future_df = pd.read_csv('future.csv')
future_df.head()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
0,1/1/2021,NaN,0,0,0,5.00,154.221
1,1/2/2021,NaN,0,0,0,11.11,264.805
2,1/3/2021,NaN,0,0,0,3.89,115.499
3,1/4/2021,NaN,0,0,0,6.67,124.650
4,1/5/2021,NaN,0,0,0,5.56,77.968


In [12]:
# merge dfs

df = pd.concat([df, future_df])
df = df.reset_index(drop=True)
df.tail()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
2280,1/27/2021,NaN,0,0,0,3.33,39.664
2281,1/28/2021,NaN,0,0,0,1.67,195.314
2282,1/29/2021,NaN,0,0,0,-2.78,235.894
2283,1/30/2021,NaN,0,0,0,1.11,152.752
2284,1/31/2021,NaN,0,0,0,4.44,158.620


In [13]:
#Rename variable
df = df.rename(columns = {'Demand': 'y'})
df.head(0)

,Date,y,Easter,Thanksgiving,Christmas,Temperature,Marketing


## Silverkite Prep

In [14]:
# time series name specifications
metadata = MetadataParam(
    time_col="Date",
    value_col="y",
    freq="D",
    train_end_date=pd.to_datetime("2020-12-31"),
)
metadata

MetadataParam(anomaly_info=None, date_format=None, freq='D', time_col='Date', train_end_date=Timestamp('2020-12-31 00:00:00'), value_col='y')

In [15]:
# growth term possibilities
growth =  dict(
    growth_terms=["linear", "quadratic", "sqrt"]
)
growth

{'growth_terms': ['linear', 'quadratic', 'sqrt']}

In [18]:
# seasonalities
seasonality = dict(
    yearly_seasonality="auto",
    quarterly_seasonality="auto",
    monthly_seasonality="auto",
    weekly_seasonality="auto",
    daily_seasonality="auto",
)
seasonality

{'yearly_seasonality': 'auto',
 'quarterly_seasonality': 'auto',
 'monthly_seasonality': 'auto',
 'weekly_seasonality': 'auto',
 'daily_seasonality': 'auto'}

In [21]:
# holidays modeling
get_available_holiday_lookup_countries(["US"])
get_available_holidays_across_countries(countries=["US"],
                                        year_start=2015,
                                        year_end=2021)

['Christmas Day',
 'Christmas Day (Observed)',
 'Columbus Day',
 'Halloween',
 'Independence Day',
 'Independence Day (Observed)',
 'Juneteenth National Independence Day',
 'Juneteenth National Independence Day (Observed)',
 'Labor Day',
 'Martin Luther King Jr. Day',
 'Memorial Day',
 "New Year's Day",
 "New Year's Day (Observed)",
 'Thanksgiving',
 'Veterans Day',
 'Veterans Day (Observed)',
 "Washington's Birthday"]

In [24]:
# specifying events
events = dict(
    holidays_to_model_seperately = ["New Year's Day"],
    holidays_pre_num_days = 2,
    holidays_post_num_days = 2,
    holiday_pre_post_num_dict = {"New Year's Day":(3,1)},
    # adding one thats not there
    # daily_event_df_dict = {"elections": pd.DataFrame({
    #     "date": ["2016-11-03", "2020-11-03"],
    #     "event_name": ["elections"] * 2
    # })}
)
events

{'holidays_to_model_seperately': ["New Year's Day"],
 'holidays_pre_num_days': 2,
 'holidays_post_num_days': 2,
 'holiday_pre_post_num_dict': {"New Year's Day": (3, 1)}}